<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_5_tuning_bigtweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1292      0 --:--:-- --:--:-- --:--:--  1298
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_bigtweet_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.115017,4.275314,0.261512,04:06


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.060978,3.747154,0.325415,04:00


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.741475,3.708772,0.331569,04:00


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.699124,3.671965,0.336735,04:03


In [0]:
learn.save_encoder('bigtweet_fold5_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.565508,4.335676,0.333333,01:13


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.823566,4.008298,0.363985,01:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.582000,3.819418,0.376756,03:38


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.382583,3.660077,0.402299,03:28
1,3.158113,3.569994,0.408685,03:26


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.150997,3.504681,0.413793,03:24
1,3.009870,3.423677,0.413793,03:26


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.993146,3.428866,0.425287,03:12
1,2.802639,3.323863,0.435504,03:25


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.836144,3.306191,0.444444,03:17
1,2.775241,3.272295,0.449553,03:29


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.728400,3.247998,0.444444,03:33
1,2.619930,3.225925,0.453384,03:26


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.594497,3.217189,0.457216,03:26
1,2.614506,3.153289,0.466156,03:37
2,2.553591,3.149904,0.463602,03:34
3,2.434560,3.153242,0.462324,03:27


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.469609,3.190291,0.459770,03:39
1,2.549242,3.171903,0.459770,03:28
2,2.459799,3.149122,0.468710,03:32
3,2.356618,3.103166,0.472542,03:33


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.365334,3.189978,0.471264,03:37
1,2.470036,3.160473,0.472542,03:18
2,2.373271,3.141152,0.477650,03:17
3,2.261238,3.091272,0.478927,03:32


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.359533,3.193397,0.472542,03:35
1,2.364964,3.182125,0.472542,03:39
2,2.359078,3.214463,0.475096,03:19
3,2.310391,3.192476,0.481481,03:18


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.216544,3.217148,0.478927,03:21
1,2.367645,3.228656,0.480204,03:19
2,2.234165,3.156948,0.476373,03:32
3,2.192859,3.178156,0.478927,03:29


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.208019,3.235199,0.472542,03:23
1,2.272570,3.183113,0.478927,03:36
2,2.193661,3.233307,0.475096,03:36
3,2.170670,3.250127,0.472542,03:32


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.160095,3.307110,0.480204,03:31
1,2.252756,3.241965,0.482759,03:29
2,2.227732,3.256064,0.478927,03:26
3,2.170907,3.215430,0.481481,03:27


In [0]:
# save the best model

learn.save_encoder('ask-5-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[847, 234, 182, 182, 698, 1026, 1026, 1026, 1026, 16, 16, 16, 71, 27, 545, 27, 1035, 197, 175, 289, 864, 864, 1035, 1035, 775, 37, 1026, 48, 480, 214, 49, 49, 214, 214, 53, 155, 155, 1035, 214, 517, 517, 715, 246, 1035, 427, 129, 209, 289, 741, 289, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 111, 517, 517, 517, 80, 80, 788, 269, 521, 847, 92, 269, 517, 517, 94, 94, 94, 1035, 228, 769, 517, 98, 100, 809, 100, 298, 100, 265, 265, 28, 265, 1035, 517, 715, 289, 1035, 715, 182, 1035, 741, 730, 604, 604, 864, 182, 289, 688, 1035, 517, 517, 933, 1035, 37, 37, 125, 269, 125, 125, 125, 125, 125, 125, 135, 517, 395, 517, 226, 132, 214, 517, 1035, 865, 318, 470, 269, 517, 517, 517, 210, 289, 289, 289, 228, 864, 864, 864, 864, 864, 173, 517, 487, 210, 517, 178, 517, 517, 182, 624, 646, 183, 183, 646, 183, 183, 183, 183, 517, 240, 240, 190, 190, 604, 111, 703, 190, 190, 190, 

In [44]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

212
0.24480369515011546
